Predict Health Insurance Premiums

Dataset was retrieved from the Center for Disease Control and Prevention (CDC) website, specifically the National Health Interview Surveys (NHIS), Sample Adult Interview (https://www.cdc.gov/nchs/nhis/2022nhis.htm)

In [57]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import r2_score
     

In [58]:
df = pd.read_csv('/Users/hp/Desktop/Database Systems/FINAL PROJECT/adult22csv/adult22.csv')
df

,URBRRL,RATCAT_A,INCTCFLG_A,IMPINCFLG_A,SHOTTYPE1_A,CEVOTELC_A,CEMMETNG_A,CEVOLUN2_A,CEVOLUN1_A,HITTEST_A,...,PROXYREL_A,PROXY_A,AVAIL_A,HHSTAT_A,INTV_MON,RECTYPE,IMPNUM_A,WTFA_A,HHX,POVRATTC_A
0,2,7,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,1,1,10,1,4548.583,H059086,1.92
1,4,14,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,1,1,10,1,7087.431,H054049,10.30
2,4,14,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,1,1,10,1,8125.516,H055201,9.36
3,4,11,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,1,1,10,1,7837.390,H044893,3.66
4,1,2,0,1,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,1.0,3,1,1,10,1,10234.356,H043149,0.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27646,4,11,0,0,NaN,1.0,2.0,2.0,2.0,NaN,...,NaN,NaN,1,1,12,10,1,3599.659,H015345,3.54
27647,4,10,0,0,3.0,2.0,2.0,NaN,1.0,2.0,...,NaN,NaN,1,1,12,10,1,3220.314,H061055,3.35
27648,4,13,0,0,1.0,1.0,2.0,2.0,2.0,2.0,...,NaN,NaN,1,1,12,10,1,3198.866,H031575,4.88
27649,4,8,0,0,NaN,1.0,1.0,2.0,2.0,2.0,...,NaN,NaN,1,1,12,10,1,6207.867,H015667,2.30


In [59]:
#clean variables for linear regression: age, gender, body mass index, number of children, smoking habits, and geolocation
#reference: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9265373/

'''
SEX_A Sex of Sample Adult 
AGEP_A Age of SA (top coded)
HEIGHTTC_A Height without shoes (inches), Public Use
WEIGHTLBTC_A Weight without shoes (pounds), Public Use
BMICAT_A Categorical Body Mass Index, Public Use -> categorical variable; recalculate BMI
PCNTKIDS_A Number of children in Sample Adult family, top-coded 3+
SMKNOW_A Smoking status, now
No geolocation data

variables that can be used for insurance premium
NOTCOV_A Insurance coverage status
HICOSTR1_A Out-of-pocket premium cost - plan 2 -> there are 14039 missing values in this variable. So I tried 
dropping NOTCOV_A==2 (not covered by insurance)
HICOSTR2_A Out-of-pocket premium cost - plan 2 -> don't use this variable; mostly missing
'''

data = df[['SEX_A', 'AGEP_A', 'HEIGHTTC_A', 'WEIGHTLBTC_A', 'PCNTKIDS_A', 'SMKNOW_A', 'HICOSTR1_A', 'NOTCOV_A']]
data

,SEX_A,AGEP_A,HEIGHTTC_A,WEIGHTLBTC_A,PCNTKIDS_A,SMKNOW_A,HICOSTR1_A,NOTCOV_A
0,1,85,68,148,0,3.0,NaN,2
1,1,64,74,235,0,3.0,99999.0,2
2,2,37,69,218,0,NaN,NaN,2
3,2,72,64,240,0,NaN,NaN,2
4,2,84,66,183,0,NaN,NaN,2
...,...,...,...,...,...,...,...,...
27646,1,64,70,180,0,1.0,NaN,1
27647,2,67,66,142,0,NaN,NaN,2
27648,2,69,60,125,0,NaN,8316.0,2
27649,1,30,75,250,2,3.0,4608.0,2


In [60]:
#check missing data
data.isnull().sum()

SEX_A               0
AGEP_A              0
HEIGHTTC_A          0
WEIGHTLBTC_A        0
PCNTKIDS_A          0
SMKNOW_A        17754
HICOSTR1_A      14039
NOTCOV_A            0
dtype: int64

In [61]:
#SMKNOW_A was given to currents mokers. NaN values are non smokers
data['SMKNOW_A'].fillna(0, inplace=True)
smoke_mapping = {1: 1, 2: 1, 3:0, 0: 0}
data['smoke'] = data['SMKNOW_A'].map(smoke_mapping)

#there are 14039 missing values in HICOSTR1_A. Drop people who are not covered by insurance
data['NOTCOV_A'].value_counts()
data = data[data['NOTCOV_A'] == 2]
#data = data.reset_index()
data = data.drop(['NOTCOV_A'], axis=1)

#Recalculate bmi 
data['bmi'] = (data['WEIGHTLBTC_A'] / (data['HEIGHTTC_A'] ** 2)) * 703
data

/Users/hp/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/var/folders/bn/lylmq9k90070gp96mjyzdpfm0000gn/T/ipykernel_23857/2827821715.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['smoke'] = data['SMKNOW_A'].map(smoke_mapping)


,SEX_A,AGEP_A,HEIGHTTC_A,WEIGHTLBTC_A,PCNTKIDS_A,SMKNOW_A,HICOSTR1_A,smoke,bmi
0,1,85,68,148,0,3.0,NaN,0.0,22.500865
1,1,64,74,235,0,3.0,99999.0,0.0,30.168919
2,2,37,69,218,0,0.0,NaN,0.0,32.189456
3,2,72,64,240,0,0.0,NaN,0.0,41.191406
4,2,84,66,183,0,0.0,NaN,0.0,29.533747
...,...,...,...,...,...,...,...,...,...
27644,1,77,68,135,0,0.0,NaN,0.0,20.524438
27647,2,67,66,142,0,0.0,NaN,0.0,22.916896
27648,2,69,60,125,0,0.0,8316.0,0.0,24.409722
27649,1,30,75,250,2,3.0,4608.0,0.0,31.244444


In [64]:
#SHOULD WE DROP PEOPLE WITHOUT HEALTH INSURANCE PREMIUM FOR SUPERVISED LEARNING? 
#create a new dataframe with only the rows with health insurance premium cost
drop_data = data.dropna(subset=['HICOSTR1_A'])

#drop HICOSTR1_A == 99999 (People who don't know)
drop_data = drop_data[drop_data['HICOSTR1_A'] != 99999]
#drop_data = drop_data.reset_index()

#smoke variable has 2 Nan, drop them
drop_data = drop_data.dropna(subset=['smoke'])
drop_data

,SEX_A,AGEP_A,HEIGHTTC_A,WEIGHTLBTC_A,PCNTKIDS_A,SMKNOW_A,HICOSTR1_A,smoke,bmi
8,2,42,68,997,0,3.0,6600.0,0.0,151.576773
12,1,69,75,245,0,0.0,2600.0,0.0,30.619556
18,1,27,67,160,0,0.0,3250.0,0.0,25.056806
20,1,38,71,200,0,0.0,15600.0,0.0,27.891291
22,1,46,71,173,2,0.0,5400.0,0.0,24.125967
...,...,...,...,...,...,...,...,...,...
27638,1,42,69,200,1,3.0,936.0,0.0,29.531611
27642,1,50,69,168,1,2.0,20400.0,1.0,24.806553
27648,2,69,60,125,0,0.0,8316.0,0.0,24.409722
27649,1,30,75,250,2,3.0,4608.0,0.0,31.244444


In [65]:
# dividing dataset into train and test
x = drop_data[['SEX_A', 'AGEP_A', 'bmi','PCNTKIDS_A', 'smoke']]
y = drop_data[['HICOSTR1_A']]

#rename columns
x.rename(columns={'SEX_A':'sex', 'AGEP_A':'age','bmi':'bmi', 'PCNTKIDS_A':'children', 'SMKNOW_A':'smoke'}, inplace=True)
y.rename(columns={'HICOSTR1_A': 'premium'}, inplace=True)

# Split 20% with test_size=0.2
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(x.shape, y.shape)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(10471, 5) (10471, 1)
(8376, 5) (8376, 1)
(2095, 5) (2095, 1)


/Users/hp/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [67]:
#Linear regression
model = LinearRegression()
model.fit(X_train, y_train)
train_pred = model.predict(X_train)

# calculate the accuracy of the model by computing the R2 score between predicted and real values
r2_train = metrics.r2_score(y_train, train_pred)
print('R squared vale : ', r2_train)

R squared vale :  0.006022651523505451


In [68]:
# prediction on test data
test_pred =model.predict(X_test)

# R squared value
r2_test = metrics.r2_score(y_test, test_pred)
print('R squared vale : ', r2_test)

R squared vale :  0.003901908046167968


In [70]:
#predictive system: if user inputs client's data used for prediction, returns predicted insurance cost
input_data = (2, 31,25.74,0,1,0)

# changing input_data to a numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = model.predict(input_data_reshaped)
print(prediction)

print('The insurance cost is USD ', prediction[0])
     

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 5 is different from 6)